In [1]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np

In [2]:
with open("data/nyt/rel2id.json") as f:
    rel2id = json.load(f)

In [3]:
id2rel = {}
for key, value in rel2id.iteritems():
    id2rel[value-1] = key

In [4]:
N_realtion = len(id2rel)

In [5]:
with open("test_result/nyt_pcnn_ave_pred.json") as f:
    test_result = json.load(f)

In [6]:
test_result[0]

{u'entpair': u'/guid/9202a8c04000641f8000000003147149#/guid/9202a8c04000641f8000000000024ba1',
 u'relation': 1,
 u'score': nan}

In [7]:
np.isnan(test_result[0]["score"])

True

In [8]:
prev = ""
pair_orders = []
for entity_pair in test_result:
    if entity_pair["entpair"] != prev:
        pair_orders.append(entity_pair["entpair"])
        prev = entity_pair["entpair"]

In [9]:
pair_orders

[u'/guid/9202a8c04000641f8000000003147149#/guid/9202a8c04000641f8000000000024ba1',
 u'/guid/9202a8c04000641f800000000006daf1#/guid/9202a8c04000641f800000000001de10',
 u'/guid/9202a8c04000641f800000000002e90b#/guid/9202a8c04000641f8000000000687b53',
 u'/guid/9202a8c04000641f8000000000078105#/guid/9202a8c04000641f800000000006bea9']

In [10]:
relation_probs = {}
for entity_pair in test_result:
    key = entity_pair["entpair"]
    probs = relation_probs.get(key,np.zeros((N_realtion,)))
    if np.isnan(entity_pair["score"]):
        prob = 0
    else:
        prob = entity_pair["score"]
    probs[entity_pair["relation"]-1] = prob
    relation_probs[key] = probs

In [11]:
pred_ralations = []
for key in pair_orders:
    pred_ralations.append(id2rel[np.argmax(relation_probs[key])])

In [12]:
with open("data/nyt/test.json") as f:
    test_input = json.load(f)

In [13]:
assert (len(test_input)) == len(pred_ralations) 

In [14]:
output = []
for single_input, pred_ralation in zip(test_input, pred_ralations):
    to_append = (single_input["sentence"].encode('utf-8'), single_input["head"]["word"].encode('utf-8'), 
                 single_input["tail"]["word"].encode('utf-8'), pred_ralation, single_input["relation"].encode('utf-8'))
    output.append(to_append)

In [15]:
with open("test_result/common_output.tsv", "w") as f:
    for element in output:
        normal_str = [str(word) for word in element]
        f.write("\t".join(normal_str) + "\n")